In [1]:
import tensorflow as tf
import numpy as np

# Sample data (English-French translation)
english_sentences = ['he is a student', 'she loves coding', 'they are friends']
french_sentences = ['il est étudiant', 'elle aime coder', 'ils sont amis']

# Tokenization
english_tokenizer = tf.keras.preprocessing.text.Tokenizer()
english_tokenizer.fit_on_texts(english_sentences)
english_seq = english_tokenizer.texts_to_sequences(english_sentences)

french_tokenizer = tf.keras.preprocessing.text.Tokenizer()
french_tokenizer.fit_on_texts(french_sentences)
french_seq = french_tokenizer.texts_to_sequences(french_sentences)

# Padding sequences
english_seq_padded = tf.keras.preprocessing.sequence.pad_sequences(english_seq, padding='post')
french_seq_padded = tf.keras.preprocessing.sequence.pad_sequences(french_seq, padding='post')

# Define encoder
encoder_inputs = tf.keras.layers.Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding(len(english_tokenizer.word_index)+1, 256)(encoder_inputs)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(256, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Define decoder
decoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding(len(french_tokenizer.word_index)+1, 256)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
attention = tf.keras.layers.Attention()
context_vector = attention([decoder_outputs, encoder_outputs])
decoder_combined_context = tf.keras.layers.Concatenate(axis=-1)([decoder_outputs, context_vector])
decoder_dense = tf.keras.layers.Dense(len(french_tokenizer.word_index)+1, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

# Define model
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train model (dummy example)
model.fit([english_seq_padded, french_seq_padded[:,:-1]], np.expand_dims(french_seq_padded[:,1:], -1), epochs=10, batch_size=64)

# Save model
model.save('nmt_model.h5')





Epoch 1/10

1/1 [==============================] - 3s 3s/step - loss: 2.3027
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: 2.2697
Epoch 3/10
1/1 [==============================] - 0s 10ms/step - loss: 2.2359
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 2.1998
Epoch 5/10
1/1 [==============================] - 0s 14ms/step - loss: 2.1598
Epoch 6/10
1/1 [==============================] - 0s 16ms/step - loss: 2.1145
Epoch 7/10
1/1 [==============================] - 0s 16ms/step - loss: 2.0622
Epoch 8/10
1/1 [==============================] - 0s 17ms/step - loss: 2.0013
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 1.9303
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 1.8474


c:\Users\sarve\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
